In [1]:
import pandas as pd
import numpy as np

import psycopg2
import pymysql

from datetime import timedelta

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df = pd.read_excel("/home/jupyter-ngenia-jackline/comb_limits_new.xlsx")
#df = pd.read_excel("/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220908_multiple_products_rec.xlsx")
df = df.rename(columns={'final_21_limits':'new_final_21_limit', 'bf_final_7_limit':'new_final_7_limit', 'bf_final_1_limit':'new_final_1_limit'})

df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,mobile_number,store_number,sum_3_months_trx_val,avg_3_months_trx_val,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,bf_final_21_limit,new_final_7_limit,new_final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp,af_final_21_limit,af_final_7_limit,af_final_1_limit,af95_final_21_limit,af95_final_7_limit,af95_final_1_limit,new_final_21_limit,old_src_crdt_score
0,0,0,0,254723173634,786152,5430.000,1810.000,3700.000,2022-08-25 00:00:00,2022-09-20 00:00:00,27,4,0.150,No_rules_relaxed,2,Yes,1.000,494024,Approve,0,True,0,1,601,30,12698,9206.000,2017-12-18 00:00:00,2018-01-17 00:00:00,2019-12-31 00:00:00,2018-01-17 00:00:00,713,1,written-off_default,0,12698,2017-12-18 00:00:00,0,0.000,1738,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,2,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN
1,1,1,1,254723916436,165978,2454577.510,818192.503,1005490.230,2022-08-23 00:00:00,2022-09-21 00:00:00,30,30,1.000,No_rules_relaxed,1,Yes,1.000,20417564,Approve,0,True,0,1,601,30,20000,12800.000,2017-12-18 00:00:00,2018-01-17 00:00:00,2019-12-31 00:00:00,2018-01-17 00:00:00,713,1,written-off_default,0,20000,2017-12-18 00:00:00,0,0.000,1738,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.125,0.075,0.075,102274.063,61364.438,61364.438,0.000,61364.438,61364.438,0,0,0,1,304500,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN
2,2,2,2,254720475133,745142,463108.240,154369.413,194218.800,2022-08-22 00:00:00,2022-09-21 00:00:00,31,30,0.970,No_rules_relaxed,1,Yes,1.000,22893884,Reject,0,True,0,1,601,30,28684,0.000,2017-12-21 00:00:00,2018-01-20 00:00:00,2019-12-31 00:00:00,2018-01-20 00:00:00,710,1,written-off_default,0,28684,2017-12-21 00:00:00,0,0.000,1735,0,0,0.350,0.120,0.120,0.700,0.706,0.706,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.087,0.053,0.053,13507.324,8172.498,8172.498,0.000,8172.498,8172.498,0,0,0,1,27200,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN
3,3,3,3,254724756423,978161,1100.000,366.667,0.000,0,0,0,0,0.000,No_rules_relaxed,0,0,0.000,13261352,Approve,0,True,0,2,601,30,28600,0.000,2018-01-07 00:00:00,2018-02-06 00:00:00,2018-12-31 00:00:00,2018-02-06 00:00:00,328,1,written-off_default,0,28680,2017-12-19 00:00:00,1,0.500,1718,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN
4,4,4,4,254722990261,510261,1232666.710,410888.903,668797.530,2022-08-22 00:00:00,2022-09-21 00:00:00,31,31,1.000,relax

In [4]:
df[['new_final_21_limit', 'new_final_7_limit', 'new_final_1_limit']].sum()

new_final_21_limit    204942500
new_final_7_limit     480198200
new_final_1_limit     482168800
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60233 entries, 0 to 60232
Data columns (total 85 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Unnamed: 0.2                            60233 non-null  int64         
 1   Unnamed: 0.1                            60233 non-null  int64         
 2   Unnamed: 0                              60233 non-null  int64         
 3   mobile_number                           60233 non-null  object        
 4   store_number                            60233 non-null  int64         
 5   sum_3_months_trx_val                    60233 non-null  float64       
 6   avg_3_months_trx_val                    60233 non-null  float64       
 7   approx_30_days_trx_val                  60233 non-null  float64       
 8   most_recent_trx_date_past_30_days       60233 non-null  object        
 9   last_trx_date                           60233 non-

In [6]:
below_score = df[(df['old_src_crdt_score'] >= 0) & (df['old_src_crdt_score'] <= 477)]

below_score[['new_final_21_limit', 'new_final_7_limit', 'new_final_1_limit']].sum()

new_final_21_limit     75360700
new_final_7_limit     107378500
new_final_1_limit     107680500
dtype: int64

In [7]:
below_score.to_excel('below_score.xlsx')

In [7]:
def calc_final_7_limit(df):
    
    old_src_crdt_score = df['old_src_crdt_score']
    new_final_7_limit = df['new_final_7_limit']
    
    if old_src_crdt_score >= 0 and old_src_crdt_score <=477:
        return 0
    else:
        return new_final_7_limit
    

    
def calc_final_1_limit(df):
    
    old_src_crdt_score = df['old_src_crdt_score']
    new_final_1_limit = df['new_final_1_limit']
    
    if old_src_crdt_score >= 0 and old_src_crdt_score <=477:
        return 0
    else:
        return new_final_1_limit

In [8]:
#df['final_21_limit'] = df.apply(lambda x: calc_final_21_limit(x), axis = 1)
df['updated_final_7_limit'] = df.apply(lambda x: calc_final_7_limit(x), axis = 1)
df['updated_final_1_limit'] = df.apply(lambda x: calc_final_1_limit(x), axis = 1)

In [9]:
df[['new_final_21_limit', 'updated_final_7_limit', 'updated_final_1_limit']].sum()

new_final_21_limit       204942500
updated_final_7_limit    372819700
updated_final_1_limit    374488300
dtype: int64

In [10]:
df[(df['old_src_crdt_score'] >= 0) & (df['old_src_crdt_score'] <= 477)].shape

(14536, 87)

In [11]:
df['old_src_crdt_score'].describe()

count   18999.000
mean      389.127
std       151.768
min        -1.000
25%       418.000
50%       440.000
75%       461.000
max       545.000
Name: old_src_crdt_score, dtype: float64

In [12]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,mobile_number,store_number,sum_3_months_trx_val,avg_3_months_trx_val,approx_30_days_trx_val,most_recent_trx_date_past_30_days,last_trx_date,expected_trx_days,actual_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,IDM_recommendation,IDM_limit,is_iprs_validated,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,IDM_factor_21,IDM_factor_7,IDM_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,bf_final_21_limit,new_final_7_limit,new_final_1_limit,blacklist_flag,total_final_21_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_version,created_at,record_added_to_warehouse_on_timestamp,af_final_21_limit,af_final_7_limit,af_final_1_limit,af95_final_21_limit,af95_final_7_limit,af95_final_1_limit,new_final_21_limit,old_src_crdt_score,updated_final_7_limit,updated_final_1_limit
0,0,0,0,254723173634,786152,5430.000,1810.000,3700.000,2022-08-25 00:00:00,2022-09-20 00:00:00,27,4,0.150,No_rules_relaxed,2,Yes,1.000,494024,Approve,0,True,0,1,601,30,12698,9206.000,2017-12-18 00:00:00,2018-01-17 00:00:00,2019-12-31 00:00:00,2018-01-17 00:00:00,713,1,written-off_default,0,12698,2017-12-18 00:00:00,0,0.000,1738,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,2,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN,0,0
1,1,1,1,254723916436,165978,2454577.510,818192.503,1005490.230,2022-08-23 00:00:00,2022-09-21 00:00:00,30,30,1.000,No_rules_relaxed,1,Yes,1.000,20417564,Approve,0,True,0,1,601,30,20000,12800.000,2017-12-18 00:00:00,2018-01-17 00:00:00,2019-12-31 00:00:00,2018-01-17 00:00:00,713,1,written-off_default,0,20000,2017-12-18 00:00:00,0,0.000,1738,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.125,0.075,0.075,102274.063,61364.438,61364.438,0.000,61364.438,61364.438,0,0,0,1,304500,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN,0,0
2,2,2,2,254720475133,745142,463108.240,154369.413,194218.800,2022-08-22 00:00:00,2022-09-21 00:00:00,31,30,0.970,No_rules_relaxed,1,Yes,1.000,22893884,Reject,0,True,0,1,601,30,28684,0.000,2017-12-21 00:00:00,2018-01-20 00:00:00,2019-12-31 00:00:00,2018-01-20 00:00:00,710,1,written-off_default,0,28684,2017-12-21 00:00:00,0,0.000,1735,0,0,0.350,0.120,0.120,0.700,0.706,0.706,Band 7,Band 2,0.250,0.150,0.150,0,1,1,0.500,0.087,0.053,0.053,13507.324,8172.498,8172.498,0.000,8172.498,8172.498,0,0,0,1,27200,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN,0,0
3,3,3,3,254724756423,978161,1100.000,366.667,0.000,0,0,0,0,0.000,No_rules_relaxed,0,0,0.000,13261352,Approve,0,True,0,2,601,30,28600,0.000,2018-01-07 00:00:00,2018-02-06 00:00:00,2018-12-31 00:00:00,2018-02-06 00:00:00,328,1,written-off_default,0,28680,2017-12-19 00:00:00,1,0.500,1718,0,0,0.500,0.170,0.170,1.000,1.000,1.000,Band 1,Band 2,0.000,0.000,0.000,0,0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,1,0,0,0,0,"2022-004[2022-05-14, 2022-09-22]",2022-09-22 20:56:58,2022-09-22 20:57:04:057203,0,0,0,0,0,0,0,NaN,0,0
4,4,4,4,254722990261,510261,1232666.710,410888.903,668797.5

In [13]:
df.to_excel('/home/jupyter-ngenia-jackline/Limits_refresh_summary_20220922_multiple_products_rec_new.xlsx')

In [14]:
df.shape

(60233, 87)